In [2]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from scipy.sparse import eye
# from sklearn import metrics
from scipy.sparse.linalg import inv

In [3]:
healthy_df = pd.read_excel('features_30_9_healthy.xlsx', header=None)

In [4]:
healthy_df

,0,1,2,3,4,5,6,7,8,9,...,23,24,25,26,27,28,29,30,31,32
0,0.09242,0.58207,0.84332,1.85755,0.75574,9.51179,6.68282,3.31333,3.98427,0.32795,...,4.82094,4539.14893,9787,12757,5068,7689,1721357.125,1312.00500,1,AAL09091956__20181106__12038__CT__HEAD__0002__...
1,0.06342,0.96902,0.74959,1.93489,0.67394,6.65879,6.44094,3.35085,4.40313,0.47218,...,4.26796,2038.07898,10136,12932,8039,4893,702499.750,838.15259,1,AAL09091956__20181106__12038__CT__HEAD__0002__...
2,0.10062,0.68576,0.79684,1.68772,0.73787,9.50464,6.04032,3.17706,3.93885,0.38197,...,4.53904,4157.31250,9961,12583,5592,6991,1237559.375,1112.45642,1,AAL09091956__20181106__12038__CT__HEAD__0002__...
3,0.05449,0.74973,0.87516,3.00270,0.71925,7.70397,10.57323,3.66807,4.57908,0.39824,...,5.20089,3706.43335,8388,12233,4544,7689,2748470.500,1657.85120,1,AAL09091956__20181106__12038__CT__HEAD__0002__...
4,0.07776,0.70084,0.85559,2.42659,0.73281,8.71716,8.64065,3.42859,4.24003,0.38487,...,5.10152,4775.73877,9088,12583,3845,8738,2867137.750,1693.26245,1,AAL09091956__20181106__12038__CT__HEAD__0002__...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2103,0.06155,0.69752,0.89054,3.18606,0.74120,7.02580,11.77003,3.62602,4.42574,0.39767,...,4.90722,3821.18237,5915,10883,2129,8754,3848079.250,1961.65222,1,ZSA29101953__20170511__6862__CT__HEAD__0002__b...
2104,0.06068,0.75961,0.86736,2.86349,0.71831,7.67092,10.40712,3.60249,4.47293,0.40430,...,4.86279,4307.60449,5678,10646,1656,8990,3530869.500,1879.06079,1,ZSA29101953__20170511__6862__CT__HEAD__0002__b...
2105,0.09644,0.70322,0.85896,2.49303,0.74774,6.01145,9.14703,3.34126,4.10024,0.41432,...,4.70639,3502.35352,4495,10646,1420,9226,3042845.750,1744.37549,1,ZSA29101953__20170511__6862__CT__HEAD__0002__b...
2106,0.12647,0.66081,0.88106,2.77796,0.75613,5.16639,10.15986,3.13970,3.91218,0.39423,...,4.47165,2767.90771,4022,10410,1656,8754,3174905.250,1781.82642,1,ZSA29101953__20170511__6862__CT__HEAD__0002__b...


In [5]:
nneg = len(healthy_df)  # number of rows of healthy_df
healthy_names = healthy_df.iloc[:,32]  # all file names of healthy_df
unique_healthy_names = healthy_names.unique()  # unique file names of healthy_df
iN = healthy_names.factorize()[0]  # index of unique file names
n_files = len(unique_healthy_names)  # number of unique files

Xneg = healthy_df.iloc[:,:31]  # healthy_df feautures

In [6]:
insult_df = pd.read_excel('features_30_9_insult.xlsx', header=None)

In [7]:
insult_df

,0,1,2,3,4,5,6,7,8,9,...,24,25,26,27,28,29,30,31,32,33
0,0.06081,0.75858,0.85508,2.61725,0.71279,9.07447,9.53269,3.54796,4.38729,0.39242,...,3644.60889,7689,10486,4019,6467,1706812.750,1306.45044,0.09679,0.44111,AAL09091956__20181106__12038__CT__HEAD__0002__...
1,0.05805,0.80593,0.86047,2.88797,0.70327,8.43171,10.18995,3.56138,4.49190,0.41046,...,3413.93018,7340,10486,4019,6467,1868834.875,1367.05334,0.14702,0.57444,AAL09091956__20181106__12038__CT__HEAD__0002__...
2,0.05735,0.80487,0.86085,2.89208,0.69992,8.27271,10.48810,3.61427,4.52479,0.40255,...,3305.78662,6990,10311,3845,6466,1798897.125,1341.22974,0.19660,0.60778,AAL09091956__20181106__12038__CT__HEAD__0002__...
3,0.07138,0.60490,0.87878,2.49509,0.74577,7.20000,9.16320,3.48943,4.20273,0.32976,...,3614.73560,7340,11884,3845,8039,2424746.750,1557.15979,0.24542,0.55333,AAL09091956__20181106__12038__CT__HEAD__0002__...
4,0.06371,0.61074,0.88761,2.71714,0.74178,8.10054,9.98028,3.60307,4.32808,0.32754,...,4149.30322,8214,12058,3845,8213,2890672.500,1700.19775,0.29329,0.48111,AAL09091956__20181106__12038__CT__HEAD__0002__...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5930,0.06646,0.65228,0.89100,2.99205,0.75269,8.06304,10.62932,3.63314,4.42857,0.38109,...,4524.91455,6861,11120,2129,8991,3640547.750,1908.02197,0.55711,0.62222,ZSA29101953__20170511__6862__CT__HEAD__0002__b...
5931,0.06672,0.58668,0.89720,2.85336,0.75804,8.28487,10.18638,3.62216,4.34481,0.33563,...,4755.65234,6861,11120,1893,9227,3537324.750,1880.77771,0.45430,0.67111,ZSA29101953__20170511__6862__CT__HEAD__0002__b...
5932,0.07679,0.60400,0.87018,2.32624,0.75293,8.35847,8.05221,3.45328,4.18973,0.34140,...,4798.49463,6861,11120,1893,9227,2920900.750,1709.06421,0.34919,0.69000,ZSA29101953__20170511__6862__CT__HEAD__0002__b...
5933,0.07262,0.62511,0.87357,2.47208,0.74652,8.93944,8.51236,3.49029,4.25676,0.34790,...,5113.42432,7098,11120,1893,9227,3107574.750,1762.83142,0.24275,0.63889,ZSA29101953__20170511__6862__CT__HEAD__0002__b...


In [8]:
insult_32_feature = insult_df.iloc[:,31]  # вероятности больных по расст от центра
insult_33_feature = insult_df.iloc[:,32]  # вероятности больных по пересеч с маской

In [9]:
insult_33_feature.where(insult_33_feature<0.95, other=1, inplace=True)
big_prob_33_feature = insult_33_feature > 0.05  # there is no feature less than 0.05
insult_32_feature = insult_32_feature[big_prob_33_feature]
insult_33_feature = insult_33_feature[big_prob_33_feature]

In [10]:
Y_pos_0 = pd.concat([insult_33_feature, pd.Series(np.zeros((nneg,), dtype=int))], axis=0)  # вероятности больных

In [11]:
Y_pos_0  # вероятности больных

0       0.44111
1       0.57444
2       0.60778
3       0.55333
4       0.48111
         ...   
2103    0.00000
2104    0.00000
2105    0.00000
2106    0.00000
2107    0.00000
Length: 8043, dtype: float64

In [12]:
insult_names = insult_df.iloc[:,33]  # all file names of insult_df
insult_names = insult_names[big_prob_33_feature]

Xpos = insult_df[big_prob_33_feature].iloc[:,:31]  # features of insult_df
npos = len(Xpos)

In [13]:
unique_insult_names = insult_names.unique()  # unique file names in insult_df
iP = insult_names.factorize()[0]  # index of unique file names

In [14]:
indexes = np.concatenate([iP, iN], axis=0)
X0 = pd.concat([Xpos, Xneg], axis=0)

In [15]:
indexes

array([ 0,  0,  0, ..., 45, 45, 45], dtype=int64)

In [16]:
K = 2

In [17]:
mu = X0.mean()
sig = X0.std()
X0 = (X0 - mu)/sig

In [18]:
X0

,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
0,-0.658769,-0.388568,0.998650,0.893271,0.168592,1.007406,0.996314,1.102418,0.480440,-0.489884,...,-0.333025,-1.016576,0.676326,-1.419898,-1.606353,-1.792862,-0.994495,-1.588028,-1.001290,-1.372388
1,-0.768904,-0.176755,1.068378,1.325875,-0.045791,0.465373,1.266773,1.169118,0.840055,-0.277057,...,-0.589243,-1.096587,0.822323,-1.452464,-1.638076,-1.792862,-0.994495,-1.588028,-0.995633,-1.349013
2,-0.796837,-0.181497,1.073294,1.332442,-0.121230,0.331290,1.389460,1.431993,0.953120,-0.370375,...,-0.616231,-1.059719,0.721063,-1.467731,-1.669890,-1.803664,-1.017111,-1.588102,-0.998075,-1.358973
3,-0.236982,-1.076033,1.305248,0.698062,0.911277,-0.573315,0.844271,0.811511,-0.154018,-1.229118,...,-0.706493,-1.333484,1.291670,-1.424115,-1.638076,-1.706570,-1.017111,-1.471346,-0.976223,-1.275687
4,-0.543047,-1.049908,1.419478,1.052893,0.821425,0.186101,1.180495,1.376327,0.276895,-1.255308,...,-0.629795,-1.619799,1.687822,-1.348647,-1.558632,-1.695829,-1.017111,-1.458430,-0.959956,-1.220516
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2103,-0.629240,-0.661711,1.457382,1.802215,0.808364,-0.720216,1.916966,1.490393,0.612618,-0.427947,...,-0.879920,-1.132671,0.862539,-1.394970,-1.767604,-1.768357,-1.240160,-1.418275,-0.926528,-1.119671
2104,-0.663956,-0.383961,1.157511,1.286756,0.292898,-0.176193,1.356137,1.373444,0.774842,-0.349730,...,-0.723328,-0.971079,0.746437,-1.326298,-1.789147,-1.782986,-1.301641,-1.400757,-0.937603,-1.151527
2105,0.763015,-0.636213,1.048844,0.694770,0.955640,-1.575607,0.837618,0.075074,-0.506346,-0.231518,...,-0.857973,-0.356089,0.337741,-1.439981,-1.896678,-1.782986,-1.332317,-1.383240,-0.954642,-1.203477
2106,1.961335,-0.825928,1.334743,1.150081,1.144576,-2.288238,1.254391,-0.926722,-1.152835,-0.468531,...,-0.981808,0.668371,-0.275668,-1.543667,-1.939672,-1.797553,-1.301641,-1.418275,-0.950032,-1.189031


In [19]:
n = npos + nneg
Y1M0 = pd.concat([insult_33_feature, pd.Series(np.zeros((nneg,), dtype=int))], axis=0)  # вероятности больных по маскам

In [20]:
# quality metrics
qual_1 = np.zeros((5,), dtype=int)
qual_2 = np.zeros((5,), dtype=int)

In [21]:
n_certain_files = 0
n_uncertain_files = n_files - 1
n_test_files = n_files - n_uncertain_files - n_certain_files

In [22]:
from sklearn.cluster import KMeans
from scipy.sparse import hstack

def clus_en_matrix(X, L, K):
    n = len(X)
    c_mem = np.zeros((n,L), dtype=int)
    A_m = [None]*L
    K_l = np.zeros(L, dtype=int)
    D = csr_matrix((n,n), dtype='int')
#     opts = ...
    for l in range(L):
        C = KMeans(n_clusters=K+l, random_state=0, max_iter=100).fit(X)
        c_mem[:,l] = C.labels_
        
    for l in range(L):
        A_m[l] = csr_matrix((np.ones(n, dtype='int'),
                    (np.arange(n),c_mem[:,l])), shape=(n,K+l))
        
    A = A_m[1]
    for l in range(1,L):
        A = hstack([A, A_m[l]])
    
    A = np.sqrt(1/L)*A
    return A


In [23]:
pd.set_option('display.max_columns', 6)

In [ ]:
for n_file in range(n_files):
    inp_files = [i for i in range(n_files) if i!=n_file] + [n_file]
    inp = []
    for j in range(n_files):
        ind_files = [i for i, x in enumerate(indexes) if x==inp_files[j]]  # all indexes with number 1,2,3,...,46,1
        inp.extend(ind_files)  # concatenate indexes in loop
    new_indexes = indexes[inp]  # новый порядок объектов
    
    # какие объекы в последних контрольных снимках
    tes_test = np.isin(indexes, inp_files[len(inp_files)-n_test_files:len(inp_files)])  # inp_files[46:46]
    n_test_0 = sum(tes_test)  # количество объектов в контрольной выборке = 277
        
    # какие объекы в неточных снимках
    tes_uncertain = np.isin(indexes, inp_files[n_certain_files:n_certain_files+n_uncertain_files])  # 1,2,...,45
    n_uncertain = sum(tes_uncertain)  # количество объектов в неточной выборке = 7766
        
    # какие объекы в точных снимках
    tes_certain = np.isin(indexes, inp_files[:n_certain_files])  # empty
    n_certain = sum(tes_certain)  # количество объектов в точной выборке = 0
        
    X = X0.iloc[inp]
    Y_pos = Y_pos_0.iloc[inp]
    Y1M = Y1M0.iloc[inp]
    
    ind_insult = Y1M==1  # уверенно классиф как больные
    ind_healthy = Y1M==0  # уверенно классиф как здоровые
    ind_certain = np.logical_or(ind_insult,ind_healthy)  # 4958 уверенно классифицированных данных
    
    Y10Tr = np.array([[*ind_insult[:n_certain], *Y_pos[n_certain:n_certain+n_uncertain], *pd.Series(np.zeros((n_test_0,), dtype=int))],
                      [*ind_healthy[:n_certain], *(1-Y_pos[n_certain:n_certain+n_uncertain]), *pd.Series(np.zeros((n_test_0,), dtype=int))]]).T
    
    Y1MC = Y1M[n-n_test_0:len(Y1M)]  # 8043-277 : 8043 (last)
    Y_true_test = 1+(Y1MC[ind_certain[n-n_test_0:len(ind_certain)]]==0)
    
#     Y10Tr2 = Y10Tr
#     ind_uncertain = np.logical_not(ind_certain)
#     Y10Tr2[ind_uncertain][n_certain:n-n_test_0, :] = 0
    
    #print(Y10Tr2)
    
    sigma = 1.25
    bet = 0.001
    mmax = 1000
    G = bet * eye(n, dtype='int') + csr_matrix(([*pd.to_numeric(ind_certain[:n_certain], errors='coerce'),
                 *np.ones(n_uncertain, dtype='int'),
                 *np.zeros(n_test_0, dtype='int')],
                (np.arange(n),np.arange(n))), shape=(n,n))
    G2 = bet * eye(n, dtype='int') + csr_matrix(([*pd.to_numeric(ind_certain[:n-n_test_0], errors='coerce'),
                  *np.zeros(n_test_0, dtype='int')],
                (np.arange(n),np.arange(n))), shape=(n,n))
    
    A = clus_en_matrix(X,10,100)
    m = A.shape[1]  # 1046 in order of 1055
    u = A.sum(axis = 0)  # by columns
    dd = A * u.conj().transpose()  # 8043x1 array
    dd[dd<0] = 0
    D = csr_matrix(([item for sublist in dd.tolist() for item in sublist], 
                    (np.arange(n), np.arange(n))), shape=(n,n), dtype='int')
    D = csr_matrix((1/np.sqrt(D.diagonal()),
                    (np.arange(n), np.arange(n))), shape=(n,n))
    U = D*A
    Gm = csr_matrix((1/G.diagonal(),
                    (np.arange(n), np.arange(n))), shape=(n,n))
    A1 = bet * Gm * U
    A2 = eye(m, dtype='int') - bet*(U.conj().transpose() * Gm)*U

    F = np.empty(shape=[n,K]) 
    for k in range(K):
        A3 = inv(A2) * (U.conj().transpose() * Gm * Y10Tr[:,k])
        F[:,k] = Gm * Y10Tr[:,k] + A1 * A3
        
    F = F/F.sum(axis = 1).reshape((n,1))  # нормировка
    Ff = F[n-n_test_0:,]
    Ft = Ff[ind_certain[n-n_test_0:],]

    class11 = Ft[Y_true_test==2, 0]  # negative
    class21 = Ft[Y_true_test==1, 0]  # positive
    roc_curve(class11, class21, 1)
    print(n_file)



C:\Users\Kunasyl.DESKTOP-N9JUTUQ\anaconda3\lib\site-packages\scipy\sparse\linalg\dsolve\linsolve.py:318: SparseEfficiencyWarning: splu requires CSC matrix format
  warn('splu requires CSC matrix format', SparseEfficiencyWarning)
C:\Users\Kunasyl.DESKTOP-N9JUTUQ\anaconda3\lib\site-packages\scipy\sparse\linalg\dsolve\linsolve.py:215: SparseEfficiencyWarning: spsolve is more efficient when sparse b is in the CSC matrix format
  warn('spsolve is more efficient when sparse b '



 ROC CURVE PARAMETERS:

Threshold: 0.7846260833823926
Distance: [0.26544164]
Sensitivity: 0.8161764705882353
Specificity: 0.8085106382978723
Accuracy: 0.8142076502732241
PPV: 0.925
NPV: 0.6031746031746031
FNR: 0.18382352941176472
FPR: 0.19148936170212766
FDR: 0.075
FOR: 0.3968253968253968
F1 score: 0.8671875
TP: 111
FP: 9
FN: 25
TN: 38
0


C:\Users\Kunasyl.DESKTOP-N9JUTUQ\anaconda3\lib\site-packages\scipy\sparse\linalg\dsolve\linsolve.py:318: SparseEfficiencyWarning: splu requires CSC matrix format
  warn('splu requires CSC matrix format', SparseEfficiencyWarning)
C:\Users\Kunasyl.DESKTOP-N9JUTUQ\anaconda3\lib\site-packages\scipy\sparse\linalg\dsolve\linsolve.py:215: SparseEfficiencyWarning: spsolve is more efficient when sparse b is in the CSC matrix format
  warn('spsolve is more efficient when sparse b '



 ROC CURVE PARAMETERS:

Threshold: 0.6806615058180263
Distance: [0.08698263]
Sensitivity: 0.9409090909090909
Specificity: 0.9361702127659575
Accuracy: 0.9400749063670412
PPV: 0.9857142857142858
NPV: 0.7719298245614035
FNR: 0.05909090909090909
FPR: 0.06382978723404255
FDR: 0.014285714285714285
FOR: 0.22807017543859648
F1 score: 0.9627906976744186
TP: 207
FP: 3
FN: 13
TN: 44
1


C:\Users\Kunasyl.DESKTOP-N9JUTUQ\anaconda3\lib\site-packages\scipy\sparse\linalg\dsolve\linsolve.py:318: SparseEfficiencyWarning: splu requires CSC matrix format
  warn('splu requires CSC matrix format', SparseEfficiencyWarning)
C:\Users\Kunasyl.DESKTOP-N9JUTUQ\anaconda3\lib\site-packages\scipy\sparse\linalg\dsolve\linsolve.py:215: SparseEfficiencyWarning: spsolve is more efficient when sparse b is in the CSC matrix format
  warn('spsolve is more efficient when sparse b '


...


 ROC CURVE PARAMETERS:

Threshold: 0.6875053383885582
Distance: [0.5508201]
Sensitivity: 0.8333333333333334
Specificity: 0.475
Accuracy: 0.6285714285714286
PPV: 0.5434782608695652
NPV: 0.7916666666666666
FNR: 0.16666666666666666
FPR: 0.525
FDR: 0.45652173913043476
FOR: 0.20833333333333334
F1 score: 0.6578947368421053
TP: 25
FP: 21
FN: 5
TN: 19
8


C:\Users\Kunasyl.DESKTOP-N9JUTUQ\anaconda3\lib\site-packages\scipy\sparse\linalg\dsolve\linsolve.py:318: SparseEfficiencyWarning: splu requires CSC matrix format
  warn('splu requires CSC matrix format', SparseEfficiencyWarning)
C:\Users\Kunasyl.DESKTOP-N9JUTUQ\anaconda3\lib\site-packages\scipy\sparse\linalg\dsolve\linsolve.py:215: SparseEfficiencyWarning: spsolve is more efficient when sparse b is in the CSC matrix format
  warn('spsolve is more efficient when sparse b '



 ROC CURVE PARAMETERS:

Threshold: 0.5931935383875353
Distance: [0.1502313]
Sensitivity: 0.9166666666666666
Specificity: 0.875
Accuracy: 0.8833333333333333
PPV: 0.6470588235294118
NPV: 0.9767441860465116
FNR: 0.08333333333333333
FPR: 0.125
FDR: 0.35294117647058826
FOR: 0.023255813953488372
F1 score: 0.7586206896551724
TP: 11
FP: 6
FN: 1
TN: 42
9


C:\Users\Kunasyl.DESKTOP-N9JUTUQ\anaconda3\lib\site-packages\scipy\sparse\linalg\dsolve\linsolve.py:318: SparseEfficiencyWarning: splu requires CSC matrix format
  warn('splu requires CSC matrix format', SparseEfficiencyWarning)
C:\Users\Kunasyl.DESKTOP-N9JUTUQ\anaconda3\lib\site-packages\scipy\sparse\linalg\dsolve\linsolve.py:215: SparseEfficiencyWarning: spsolve is more efficient when sparse b is in the CSC matrix format
  warn('spsolve is more efficient when sparse b '



 ROC CURVE PARAMETERS:

Threshold: 0.6791671768776798
Distance: [0.3675821]
Sensitivity: 0.7222222222222222
Specificity: 0.7592592592592593
Accuracy: 0.75
PPV: 0.5
NPV: 0.8913043478260869
FNR: 0.2777777777777778
FPR: 0.24074074074074073
FDR: 0.5
FOR: 0.10869565217391304
F1 score: 0.5909090909090909
TP: 13
FP: 13
FN: 5
TN: 41
10


In [1]:
n_file = 0
inp_files = [i for i in range(n_files) if i!=n_file] + [n_file]
inp = []
for j in range(n_files):
    ind_files = [i for i, x in enumerate(indexes) if x==inp_files[j]]  # all indexes with number 1,2,3,...,46,1
    inp.extend(ind_files)  # concatenate indexes in loop
new_indexes = indexes[inp]  # новый порядок объектов

# какие объекы в последних контрольных снимках
tes_test = np.isin(indexes, inp_files[len(inp_files)-n_test_files:len(inp_files)])  # inp_files[46:46]
n_test_0 = sum(tes_test)  # количество объектов в контрольной выборке = 277

# какие объекы в неточных снимках
tes_uncertain = np.isin(indexes, inp_files[n_certain_files:n_certain_files+n_uncertain_files])  # 1,2,...,45
n_uncertain = sum(tes_uncertain)  # количество объектов в неточной выборке = 7766

# какие объекы в точных снимках
tes_certain = np.isin(indexes, inp_files[:n_certain_files])  # empty
n_certain = sum(tes_certain)  # количество объектов в точной выборке = 0

X = X0.iloc[inp]
Y_pos = Y_pos_0.iloc[inp]
Y1M = Y1M0.iloc[inp]

ind_insult = Y1M==1  # уверенно классиф как больные
ind_healthy = Y1M==0  # уверенно классиф как здоровые
ind_certain = np.logical_or(ind_insult,ind_healthy)  # 4958 уверенно классифицированных данных

Y10Tr = np.array([[*ind_insult[:n_certain], *Y_pos[n_certain:n_certain+n_uncertain], *pd.Series(np.zeros((n_test_0,), dtype=int))],
                  [*ind_healthy[:n_certain], *(1-Y_pos[n_certain:n_certain+n_uncertain]), *pd.Series(np.zeros((n_test_0,), dtype=int))]]).T

Y1MC = Y1M[n-n_test_0:len(Y1M)]  # 8043-277 : 8043 (last)
Y_true_test = 1+(Y1MC[ind_certain[n-n_test_0:len(ind_certain)]]==0)

# Y10Tr2 = Y10Tr
# ind_uncertain = np.logical_not(ind_certain)
# Y10Tr2[ind_uncertain][n_certain:n-n_test_0, :] = 0

F = y_pred

class11 = Ft[Y_true_test==2, 1]  # negative
class21 = Ft[Y_true_test==1, 1]  # positive

NameError: name 'n_files' is not defined

In [63]:
len(Y10Tr2[ind_uncertain][n_certain:n-n_test_0, :])  #should be 3001

3085

In [65]:
len(Y10Tr2)

8043

In [64]:
Y10Tr2[ind_uncertain[n_certain:n-n_test_0],:]

IndexError: boolean index did not match indexed array along dimension 0; dimension is 8043 but corresponding boolean dimension is 7766

In [66]:
ind_uncertain[n_certain:n-n_test_0]

230      True
231      True
232      True
233      True
234      True
        ...  
2103    False
2104    False
2105    False
2106    False
2107    False
Length: 7766, dtype: bool

In [68]:
Y10Tr

array([[0.41111, 0.58889],
       [0.39444, 0.60556],
       [0.44889, 0.55111],
       ...,
       [0.     , 0.     ],
       [0.     , 0.     ],
       [0.     , 0.     ]])

In [74]:
def roc_curve(class_1, class_2, disp_data):  # negative, positive
#     rng(1337,'twister')
#     s = rng
#     class_1 = rand(1,3);
#     rng(s)
#     class_2 = 0.5*rand(1,5);
#     sispt = 1;
#     dispp = 1;

#     np.random.seed(1337)
#     class_1 = np.random.rand(1,3)
#     np.random.seed(1337)
#     class_2 = 0.5*np.random.rand(1,3)

    # Calculating the threshold values between the data points
    s_data = np.sort(np.unique(np.concatenate([class_1, class_2], axis=0)))
    s_data = s_data[s_data!=np.array(None)]
    d_data = np.diff(s_data)
    if np.all(d_data==0):
        print('Both class are the same!')
    d_data = np.append(d_data, d_data[-1])
    thres = np.empty(len(s_data)+1)
    thres[0] = s_data[0] - d_data[0]
    thres[1:len(thres)] = s_data + d_data/2

    # Calculating the sensibility and specificity of each threshold
    curve = np.zeros((len(thres),2))
    distance = np.zeros((len(thres),1))
    for id_t in range(len(thres)):
        TP = np.sum(class_2 >= thres[id_t])
        FP = np.sum(class_1 >= thres[id_t])
        FN = np.sum(class_2 < thres[id_t])
        TN = np.sum(class_1 < thres[id_t])

        curve[id_t, 0] = TP/(TP+FN)  # sensitivity - TPR
        curve[id_t, 1] = TN/(TN+FP)  # specificity - FPR

        distance[id_t] = np.sqrt((1-curve[id_t, 0])**2 + (curve[id_t,1]-1)**2)

    # Optimum threshold and parameters
    #opt = np.min(distance)  # 1
    opt = np.argmin(distance)
    TP = np.sum(class_2 >= thres[opt])
    FP = np.sum(class_1 >= thres[opt])
    FN = np.sum(class_2 < thres[opt])
    TN = np.sum(class_1 < thres[opt])

    if disp_data==1:
        print('\n ROC CURVE PARAMETERS:\n')
        print('Threshold:', thres[opt])
        print('Distance:', distance[opt])
        print('Sensitivity:', curve[opt,0])
        print('Specificity:', curve[opt,1])
        print('Accuracy:', (TP+TN)/(TP+TN+FP+FN))
        print('PPV:', TP/(TP+FP))
        print('NPV:', TN/(TN+FN))
        print('FNR:', FN/(FN+TP))
        print('FPR:', FP/(FP+TN))
        print('FDR:', FP/(FP+TP))
        print('FOR:', FN/(FN+TN))
        print('F1 score:', 2*TP/(2*TP+FP+FN))
        print('TP:', TP)
        print('FP:', FP)
        print('FN:', FN)
        print('TN:', TN)